#Speech Emotion Recognition with MLP Classifier



#Dataset
The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS) 

---
Audio-only files

Audio-only files of all actors (01-24) are available as two separate zip files (~200 MB each):

Speech file (Audio_Speech_Actors_01-24.zip, 215 MB) contains 1440 files: 60 trials per actor x 24 actors = 1440. 
Song file (Audio_Song_Actors_01-24.zip, 198 MB) contains 1012 files: 44 trials per actor x 23 actors = 1012.

Total=2452

---

---
Toronto emotional speech set (TESS)

---


There are a set of 200 target words were spoken in the carrier phrase "Say the word _' by two actresses (aged 26 and 64 years) and recordings were made of the set portraying each of seven emotions (anger, disgust, fear, happiness, pleasant surprise, sadness, and neutral). There are 2800 data points (audio files) in total.

The dataset is organised such that each of the two female actor and their emotions are contain within its own folder. And within that, all 200 target words audio file can be found. The format of the audio file is a WAV format


---



# Mount google drive



# Install following libraries

In [19]:
!pip install librosa soundfile numpy sklearn pyaudio

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\roopa\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\roopa\\AppData\\Local\\Temp\\pip-install-vy1g1bry\\pyaudio\\setup.py'"'"'; __file__='"'"'C:\\Users\\roopa\\AppData\\Local\\Temp\\pip-install-vy1g1bry\\pyaudio\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\roopa\AppData\Local\Temp\pip-wheel-ixnvcj68'
       cwd: C:\Users\roopa\AppData\Local\Temp\pip-install-vy1g1bry\pyaudio\
  Complete output (9 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  copying src\pyaudio.py -> build\lib.win-amd64-3.8
  running build_ext
  building '_portaudio' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https


  Using cached PyAudio-0.2.11.tar.gz (37 kB)
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio: started
    Running setup.py install for pyaudio: finished with status 'error'


In [21]:
!pip install soundfile

# Make the necessary imports

In [22]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

Define a function extract_feature to extract the mfcc, chroma, and mel features from a sound file. This function takes 4 parameters- the file name and three Boolean parameters for the three features:

* mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound
* chroma: Pertains to the 12 different pitch classes
* mel: Mel Spectrogram Frequency

In [23]:
def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    if chroma:
        stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

Now, let’s define a dictionary to hold numbers and the emotions available in the RAVDESS & TESS dataset, and a list to hold all 8 emotions- neutral,calm,happy,sad,angry,fearful,disgust,surprised.

In [24]:
# Emotions in the RAVDESS & TESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
# Emotions to observe
observed_emotions=['neutral','calm','happy','sad','angry','fearful']

# Load the data and extract features for each sound file

In [25]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob('./NewDataSet/*_*/*.wav'):
        file_name=os.path.basename(file)
        if file_name[0]=='0':
            emotion=emotions[file_name.split("-")[2]]
        else:
            emotion=file_name.split("_")[2]
            emotion=emotion.split(".")[0]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, train_size= 0.75,random_state=9)

# Split the Dataset
Time to split the dataset into training and testing sets! Let’s keep the test set 25% of everything and use the load_data function for this.

In [26]:
# Split the dataset
import time
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

#Observe the shape of the training and testing datasets:

In [11]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(2751, 917)


# Number of features extracted.

In [12]:
# Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


# MLP Classifier

In [13]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

#Fit/train the model.

In [14]:
# Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

# Predict the accuracy of our model

Let’s predict the values for the test set. This gives us y_pred (the predicted emotions for the features in the test set).

In [15]:
# Predict for the test set
y_pred=model.predict(x_test)

To calculate the accuracy of our model, we’ll call up the accuracy_score() function we imported from sklearn. Finally, we’ll round the accuracy to 2 decimal places and print it out.

In [16]:
# Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))
print(y_pred)

Accuracy: 81.24%
['angry' 'happy' 'angry' 'fearful' 'sad' 'angry' 'fearful' 'fearful'
 'neutral' 'angry' 'angry' 'angry' 'happy' 'happy' 'calm' 'fearful' 'calm'
 'fearful' 'fearful' 'sad' 'fearful' 'sad' 'angry' 'angry' 'sad' 'fearful'
 'fearful' 'sad' 'neutral' 'calm' 'sad' 'sad' 'happy' 'neutral' 'happy'
 'neutral' 'sad' 'sad' 'angry' 'fearful' 'calm' 'sad' 'fearful' 'angry'
 'sad' 'sad' 'neutral' 'angry' 'neutral' 'happy' 'neutral' 'fearful'
 'happy' 'fearful' 'angry' 'neutral' 'sad' 'happy' 'sad' 'happy' 'neutral'
 'sad' 'sad' 'sad' 'sad' 'sad' 'happy' 'happy' 'happy' 'neutral' 'angry'
 'neutral' 'angry' 'fearful' 'fearful' 'sad' 'sad' 'neutral' 'fearful'
 'neutral' 'sad' 'happy' 'happy' 'neutral' 'sad' 'neutral' 'angry' 'happy'
 'sad' 'calm' 'sad' 'angry' 'calm' 'sad' 'fearful' 'happy' 'sad' 'angry'
 'calm' 'calm' 'angry' 'happy' 'calm' 'neutral' 'sad' 'calm' 'neutral'
 'sad' 'sad' 'fearful' 'neutral' 'angry' 'fearful' 'neutral' 'happy'
 'angry' 'fearful' 'neutral' 'fearful' 'sad'

#classification Report

In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))


              precision    recall  f1-score   support

       angry       0.90      0.86      0.88       177
        calm       0.88      0.57      0.69        98
     fearful       0.51      0.78      0.62        83
       happy       0.90      0.81      0.85       193
     neutral       0.94      0.85      0.89       154
         sad       0.76      0.86      0.81       212

    accuracy                           0.81       917
   macro avg       0.81      0.79      0.79       917
weighted avg       0.84      0.81      0.82       917



# Confusion Matrix

In [18]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test,y_pred)
print (matrix)

[[153   0  13   6   1   4]
 [  3  56   6   7   4  22]
 [  6   0  65   1   0  11]
 [  5   2  19 157   0  10]
 [  2   4   3   3 131  11]
 [  1   2  21   1   4 183]]


#Thank You